In [1]:
import sys
import os, copy
import torch
import matplotlib.pyplot as plt
import numpy as np
import re
from collections import OrderedDict
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


/home/qasim/miniconda3/envs/1xgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Model conversion utils

In [4]:
def state_dict_to_vector(state_dict, remove_keys=[]):
    shared_state_dict = copy.deepcopy(state_dict)
    for key in remove_keys:
        if key in shared_state_dict:
            del shared_state_dict[key]
    sorted_shared_state_dict = OrderedDict(sorted(shared_state_dict.items()))
    return torch.nn.utils.parameters_to_vector(
        [value.reshape(-1) for key, value in sorted_shared_state_dict.items()]
    )


def vector_to_state_dict(vector, state_dict, remove_keys=[]):
    # create a reference dict to define the order of the vector
    reference_dict = copy.deepcopy(state_dict)
    for key in remove_keys:
        if key in reference_dict:
            del reference_dict[key]
    sorted_reference_dict = OrderedDict(sorted(reference_dict.items()))

    # create a shared state dict using the refence dict
    torch.nn.utils.vector_to_parameters(vector, sorted_reference_dict.values())

    # add back the encoder and decoder embedding weights.
    if "transformer.shared.weight" in sorted_reference_dict:
        for key in remove_keys:
            sorted_reference_dict[key] = sorted_reference_dict[
                "transformer.shared.weight"
            ]
    return sorted_reference_dict


def add_ptm_to_tv(tv_dict, ptm_dict):
    assert set(tv_dict.keys()) == set(
        ptm_dict.keys()
    ), "Differing parameter names in models."
    final_dict = copy.deepcopy(tv_dict)
    for k, v in ptm_dict.items():
        final_dict[k] = tv_dict[k] + v
    return final_dict


def check_parameterNamesMatch(checkpoints):
    parameter_names = set(checkpoints[0].keys())

    if len(checkpoints) >= 2:
        # raise ValueError("Number of models is less than 2.")
        for checkpoint in checkpoints[1:]:
            current_parameterNames = set(checkpoint.keys())
            if current_parameterNames != parameter_names:
                raise ValueError(
                    "Differing parameter names in models. "
                    f"The different parameters are {parameter_names.symmetric_difference(current_parameterNames)}"
                )

def check_state_dicts_equal(state_dict1, state_dict2):
    if set(state_dict1.keys()) != set(state_dict2.keys()):
        return False

    for key in state_dict1.keys():
        if not torch.equal(state_dict1[key], state_dict2[key]):
            return False

    return True

#### Load base model and the Finetuned Models to merge

In [5]:
os.chdir('/pub0/qasim/1xgpt/1xgpt')
from genie.st_mask_git import STMaskGIT
import os 
import argparse
import time
import os
import sys
from collections import defaultdict
from pathlib import Path
import lpips
import torch
import transformers
from einops import rearrange
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import default_data_collator
sys.path.append(os.getcwd())
from data import RawTokenDataset
from visualize import decode_latents_wrapper
from eval_utils import decode_tokens, compute_lpips, AvgMetric, compute_loss
from genie.st_mask_git import STMaskGIT
from genie.evaluate import GenieEvaluator
from operator import itemgetter
from copy import deepcopy

pt_checkpoint = '/pub0/qasim/1xgpt/1xgpt/checkpoints/GENIE_35M'
checkpoints = [
    '/pub0/qasim/1xgpt/1xgpt/logs/train_0-10-21-14-24/step_270000',
    '/pub0/qasim/1xgpt/1xgpt/logs/train_1-10-20-23-15/step_320000',
    # '/pub0/qasim/1xgpt/1xgpt/logs/train_2-10-21-18-12/step_400000'
]
result_folder = '/pub0/qasim/1xgpt/1xgpt/checkpoints/TiesMerging'
logs_folder = os.path.join(result_folder, 'merge_tests')
os.makedirs(result_folder, exist_ok=True)
os.makedirs(logs_folder, exist_ok=True)
device = 'cuda:1'

def save_checkpoint(model, save_path):
    model.save_pretrained( save_path )

ft_checks = []
for i in range(len(checkpoints)):
    ft_checks.append(STMaskGIT.from_pretrained(checkpoints[i]).state_dict())

ptm_check = STMaskGIT.from_pretrained(pt_checkpoint).state_dict()


# Removing the two keys from state dict when creating the task vector.
# Basically these keys are not involved in the global operations like the computation of topk.
remove_keys_startswith = [
    # 'pos_embed_TSC',
    # 'token_embed',
    # 'out_x_proj'
]
remove_keys_endswith = [
    # '.norm1.weight',
    # '.norm1.bias',
    # '.norm2.weight',
    # '.norm2.bias',
    # '.proj.weight',
    # '.proj.bias',
    # '.qkv.weight',
    # '.fc1.weight',
    # '.fc1.bias',
    # '.fc2.weight',
    # '.fc2.bias',
]
remove_keys = []
for k in ptm_check.keys():
    for l in remove_keys_startswith:
        if k.startswith(l):
            remove_keys.append(k)
            break
    for l in remove_keys_endswith:
        # print(k, l, k.endswith(l))
        if k.endswith(l):
            remove_keys.append(k)
            break

# print(remove_keys)
for k in remove_keys:
    assert k in ptm_check
    ptm_check.pop(k)
    assert not (k in ptm_check)
    for m in ft_checks:
        m.pop(k)
for k in ptm_check.keys():
    print(k)

hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim

In [6]:
class GenieEvaluatorCustom(GenieEvaluator):
    def __init__(self, model, latent_h, latent_w, maskgit_steps, temperature, decode_latents, device):
        # super().__init__()
        self.model = model
        self.model = self.model.to(device=device)
        self.model.eval()
        self.decode_latents = decode_latents
        self.device = device
        self.latent_h, self.latent_w = latent_h, latent_w
        self.maskgit_steps = maskgit_steps
        self.temperature = temperature

@torch.no_grad()
def evaluate(model, log_file, device):
    transformers.set_seed(42)
    val_data_dir = "../data/val_v1.1"
    WINDOW_SIZE = 16
    val_dataset = RawTokenDataset(val_data_dir, window_size=WINDOW_SIZE, stride=15, filter_overlaps=True)
    latent_h = latent_w = val_dataset.metadata["s"]
    decode_latents = decode_latents_wrapper(device=device)
    lpips_alex = lpips.LPIPS(net="alex")  # Calculate LPIPS w/ AlexNet, which is the fastest model out of their options
    maskgit_steps = 2
    temperature = 0

    dataloader = DataLoader(val_dataset, collate_fn=default_data_collator, batch_size=6)

    evaluator = GenieEvaluatorCustom(model, latent_h, latent_w, maskgit_steps, temperature, decode_latents, device=device)
    metrics = defaultdict(AvgMetric)

    f = open(log_file, 'w')
    for batch in tqdm(dataloader):
        batch_size = batch["input_ids"].size(0)
        print(batch_size)
        reshaped_input_ids = rearrange(batch["input_ids"], "b (t h w) -> b t h w", 
            t=WINDOW_SIZE,
            h=latent_h,
            w=latent_w)

        start_time = time.time()
        samples, factored_logits = evaluator.predict_zframe_logits(batch["input_ids"])
        frames_per_batch = (WINDOW_SIZE - 1) * batch["input_ids"].size(0)
        metrics["gen_time"].update((time.time() - start_time) / frames_per_batch, batch_size)

        loss = compute_loss(batch["labels"], factored_logits)

        acc = (reshaped_input_ids[:, 1:].to(device) == samples).float().mean().item()

        metrics["loss"].update(loss, batch_size)
        metrics["acc"].update(acc, batch_size)

        start_time = time.time()
        pred_frames = evaluator.predict_next_frames(samples, device=device)
        metrics["dec_time"].update((time.time() - start_time) / frames_per_batch, batch_size)

        decoded_gtruth = decode_tokens(reshaped_input_ids, decode_latents, device=device)
        metrics["pred_lpips"].update_list(compute_lpips(decoded_gtruth[:, 1:], pred_frames, lpips_alex))
        
    for key,val in metrics.items():
        f.write(key + ': ' + str(val.mean()) + '\n')
    f.close()

    return - metrics['loss'].mean()


In [7]:
# check if all checkpoints have the same paramters.
check_parameterNamesMatch(ft_checks + [ptm_check])

print(f"Flattening out Checkpoints")
flat_ft = torch.vstack(
    [state_dict_to_vector(check, remove_keys) for check in ft_checks]
)
flat_ptm = state_dict_to_vector(ptm_check, remove_keys)

# Creating Task vectors
tv_flat_checks = flat_ft - flat_ptm

# check if the vectorized state dicts can be converted back to the original state dicts
# covnert back the flat task vectors to state dict and see if the original and converted sd's are equal
assert check_state_dicts_equal(
        vector_to_state_dict(flat_ptm, ptm_check, remove_keys), ptm_check
    )
assert all(
    [
        check_state_dicts_equal(
            vector_to_state_dict(flat_ft[i], ptm_check, remove_keys), ft_checks[i]
        )
        for i in range(len(ft_checks))
    ]
)

Flattening out Checkpoints


#### Merge Utils

In [8]:
## TIES MERGING UTILS

def topk_values_mask(M, K=0.7, return_mask=False):
    if K > 1:
        K /= 100

    original_shape = M.shape
    if M.dim() == 1:
        M = M.unsqueeze(0)

    n, d = M.shape
    k = int(d * K)
    k = d - k  # Keep top k elements instead of bottom k elements

    # Find the k-th smallest element by magnitude for each row
    kth_values, _ = M.abs().kthvalue(k, dim=1, keepdim=True)
    # Create a mask tensor with True for the top k elements in each row
    mask = M.abs() >= kth_values
    final_mask = mask.squeeze() if original_shape == M.squeeze().shape else mask

    if return_mask:
        return M * final_mask, final_mask.float().mean(dim=1), final_mask
    return M * final_mask, final_mask.float().mean(dim=1)


def resolve_zero_signs(sign_to_mult, method="majority"):
    majority_sign = torch.sign(sign_to_mult.sum())

    if method == "majority":
        sign_to_mult[sign_to_mult == 0] = majority_sign
    elif method == "minority":
        sign_to_mult[sign_to_mult == 0] = -1 * majority_sign
    return sign_to_mult


def resolve_sign(Tensor):
    sign_to_mult = torch.sign(Tensor.sum(dim=0))
    sign_to_mult = resolve_zero_signs(sign_to_mult, "majority")
    return sign_to_mult


def disjoint_merge(Tensor, merge_func, sign_to_mult):

    merge_func = merge_func.split("-")[-1]

    # If sign is provided then we select the corresponding entries and aggregate.
    if sign_to_mult is not None:
        rows_to_keep = torch.where(
            sign_to_mult.unsqueeze(0) > 0, Tensor > 0, Tensor < 0
        )
        selected_entries = Tensor * rows_to_keep
    # Else we select all non-zero entries and aggregate.
    else:
        rows_to_keep = Tensor != 0
        selected_entries = Tensor * rows_to_keep

    if merge_func == "mean":
        non_zero_counts = (selected_entries != 0).sum(dim=0).float()
        disjoint_aggs = torch.sum(selected_entries, dim=0) / torch.clamp(
            non_zero_counts, min=1
        )
    elif merge_func == "sum":
        disjoint_aggs = torch.sum(selected_entries, dim=0)
    elif merge_func == "max":
        disjoint_aggs = selected_entries.abs().max(dim=0)[0]
        disjoint_aggs *= sign_to_mult
    else:
        raise ValueError(f"Merge method {merge_func} is not defined.")

    return disjoint_aggs


def ties_merging(
    flat_task_checks,
    reset_thresh=None,
    merge_func="",
):
    all_checks = flat_task_checks.clone()
    updated_checks, *_ = topk_values_mask(
        all_checks, K=reset_thresh, return_mask=False
    )
    print(f"RESOLVING SIGN")
    final_signs = resolve_sign(updated_checks)
    assert final_signs is not None
    
    print(f"Disjoint AGGREGATION: {merge_func}")
    merged_tv = disjoint_merge(updated_checks, merge_func, final_signs)
    
    return merged_tv



#### TIES Merging

In [9]:
# TIES Merging example
K = 20
merge_func = "dis-mean"
lamda = 1.0

# return merged flat task vector
merged_tv = ties_merging(
    tv_flat_checks,
    reset_thresh=K,
    merge_func=merge_func,
)

# add back the PTM to the flat merged task vector
merged_check = flat_ptm + lamda * merged_tv

# convert the flat merged checkpoint to a state dict
merged_state_dict = vector_to_state_dict(
    merged_check, ptm_check, remove_keys=remove_keys
)

RESOLVING SIGN
Disjoint AGGREGATION: dis-mean


In [11]:
model = STMaskGIT.from_pretrained('/pub0/qasim/1xgpt/1xgpt/logs/train_1-10-29-21-28/step_60000') # checkpoints[1]

# /pub0/qasim/1xgpt/1xgpt/logs/train_1-10-26-14-09/step_10000
# model = STMaskGIT.from_pretrained(pt_checkpoint)
# model.load_state_dict(merged_state_dict, strict=False)

evaluate(model, '/pub0/qasim/1xgpt/1xgpt/checkpoints/TiesMerging/trash.txt', device)

hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim 256
hidden_dim

  0%|          | 0/41 [00:00<?, ?it/s]

6
Generating frame 1


100%|██████████| 2/2 [00:00<00:00,  7.76it/s]


Generating frame 2


...

Generating frame 14


100%|██████████| 2/2 [00:00<00:00, 14.50it/s]


Generating frame 15


100%|██████████| 41/41 [06:48<00:00,  9.96s/it]


-9.3511706104985

#### Task Vector Merging

In [7]:
## TASK VECTOR MERGING UTILS

def aggregate(T, agg_type, dim=0):
    if agg_type == "mean":
        result = torch.mean(T, dim=dim)
    elif agg_type == "sum":
        result = torch.sum(T, dim=dim)
    else:
        raise ValueError("Invalid agg_type: %s" % agg_type)

    return result

def tv_merging(tv_flat_checks):
    """Merging by creating and scaling Task Vectors"""
    all_checks = tv_flat_checks.clone()
    tv_merged_check = aggregate(all_checks, "sum")
    return tv_merged_check


In [8]:
# Task Vector Merging example
lamda = 0.4

merged_tv = tv_merging(tv_flat_checks)
merged_check = flat_ptm + lamda * merged_tv
merged_state_dict = vector_to_state_dict(
    merged_check, ptm_check, remove_keys=remove_keys
)